In [2]:
import torch
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import math
import random

In [3]:
df = pd.read_csv("/home/shreyasmn/Downloads/pilDataSets/RAMON_DATASET.csv")
df.iloc[75310,1:13].astype(bool).sum(axis=0)

12

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,rssi_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,64.328276,10.0,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0


Adding a deployment column for the dataset

In [5]:
def add_deployment_column(df):
    path = "/home/shreyasmn/Downloads/pilDataSets/RAMON_DATASET.csv"
    count = -1
    
    k=12
    i=0
    bss_num=0
    
    for idx, val in df.iterrows():
        if df.at[idx,"node_type"]==0:
            bss_num+=1
        if (bss_num-1)%k==0 and df.at[idx, "node_type"] == 0:
            count += 1
        df.at[idx, "deployment"] = count
        if bss_num==7680:
            k=10
        if bss_num==14080:
            k=8
    return df


In [6]:
df = add_deployment_column(df)

adding interference map for a given deployment

In [7]:
df.head()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7,deployment
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,10.0,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0,0.0
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0,0.0
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0,0.0
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0,0.0
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0,0.0


In [8]:
for i in range(12):
    df[str(i)] = 0

In [9]:
df.head()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,2,3,4,5,6,7,8,9,10,11
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,0,0,0,0,0,0,0,0,0,0
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,0,0,0,0,0,0,0,0,0,0
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,0,0,0,0,0,0,0,0,0,0
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,0,0,0,0,0,0,0,0,0,0
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,0,0,0,0,0,0,0,0,0,0


In [13]:
for idx, val in df.iterrows():
    s = df.at[idx, 'int_map']
    s = s.replace('[', '')
    s = s.replace(']', '')
    list_s = s.split(',')
    for i, val in enumerate(list_s):
        df.at[idx, str(i)] = val

In [15]:
df.tail()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,2,3,4,5,6,7,8,9,10,11
306419,306419,306419,STA_H7,1,H,43.8570,13.9867,7.1174,2.4,6,...,-69.33,-62.63,-87.68,-76.34,-62.51,0,0,0,0,0
306420,306420,306420,STA_H8,1,H,33.9621,21.7909,5.6933,2.4,6,...,-69.33,-62.63,-87.68,-76.34,-62.51,0,0,0,0,0
306421,306421,306421,STA_H9,1,H,42.1158,13.5457,7.3812,2.4,6,...,-69.33,-62.63,-87.68,-76.34,-62.51,0,0,0,0,0
306422,306422,306422,STA_H10,1,H,40.0639,8.4542,4.6942,2.4,6,...,-69.33,-62.63,-87.68,-76.34,-62.51,0,0,0,0,0
306423,306423,306423,STA_H11,1,H,42.7649,14.7606,5.9192,2.4,6,...,-69.33,-62.63,-87.68,-76.34,-62.51,0,0,0,0,0


In [19]:
def get_dep(dep, df):
    df_1 = df
    dep = df_1.loc[df_1['deployment'] == dep]
    dep_reset = dep.reset_index(drop=True)
    return dep_reset

,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7,deployment,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,0,-66.46,-82.23,-95.47,-67.63,-74.23,-86.04,-97.83,-84.33,-87.1,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,0,-66.46,-82.23,-95.47,-67.63,-74.23,-86.04,-97.83,-84.33,-87.1,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,0,-66.46,-82.23,-95.47,-67.63,-74.23,-86.04,-97.83,-84.33,-87.1,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,0,-66.46,-82.23,-95.47,-67.63,-74.23,-86.04,-97.83,-84.33,-87.1,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,0,-66.46,-82.23,-95.47,-67.63,-74.23,-86.04,-97.83,-84.33,-87.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,-87.1,-84.04,-87.21,-94.58,-74.2,-67.82,-74.78,-86.16,-67.7,0,0,0
163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,-87.1,-84.04,-87.21,-94.58,-74.2,-67.82,-74.78,-86.16,-67.7,0,0,0
164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,-87.1,-84.04,-87.21,-94.58,-74.2,-67.82,-74.78,-86.16,-67.7,0,0,0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,799.0,-87.1,-84.04,-87.21,-94.58,-74.2,-67.82,-74.78,-86.16,-67.7,0,0,0
